# Efficiently train Large Language Models with LoRA and Hugging Face on CM

You will learn how to:

1. Setup Development Environment
2. Load and prepare the dataset
3. Fine-Tune T5 with LoRA and bnb int-8
4. Evaluate & run Inference with LoRA FLAN-T5

### Quick intro: PEFT or Parameter Efficient Fine-tunin

[PEFT](https://github.com/huggingface/peft), or Parameter Efficient Fine-tuning, is a new open-source library from Hugging Face to enable efficient adaptation of pre-trained language models (PLMs) to various downstream applications without fine-tuning all the model's parameters. PEFT currently includes techniques for:

- LoRA: [LORA: LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS](https://arxiv.org/pdf/2106.09685.pdf)
- Prefix Tuning: [P-Tuning v2: Prompt Tuning Can Be Comparable to Fine-tuning Universally Across Scales and Tasks](https://arxiv.org/pdf/2110.07602.pdf)
- P-Tuning: [GPT Understands, Too](https://arxiv.org/pdf/2103.10385.pdf)
- Prompt Tuning: [The Power of Scale for Parameter-Efficient Prompt Tuning](https://arxiv.org/pdf/2104.08691.pdf)

*Note: This tutorial was created and run on a g4dn.12xLarge using 2 GPUs

## 1. Setup Development Environment

In our example, we use the [PyTorch Deep Learning AMI](https://docs.aws.amazon.com/dlami/latest/devguide/tutorial-pytorch.html) with already set up CUDA drivers and PyTorch installed. We still have to install the Hugging Face Libraries, including transformers and datasets. Running the following cell will install all the required packages.

In [1]:
# install Hugging Face Libraries
#!pip install  peft==0.2.0 datasets==2.14.5 transformers==4.27.1 accelerate==0.17.1 evaluate==0.4.0 bitsandbytes==0.37.1 pandas rouge-score tensorboard py7zr s3fs loralib --upgrade

In [2]:
#!pip install s3fs

In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# !cp '/content/gdrive/My Drive/Data/FINDSum/text/FINDSum-ROO/roo_input_2000/'* .

In [4]:
# Let us get Dataload setup
import os
import pandas as pd
import datasets
import s3fs
from s3fs import S3FileSystem
from datasets import Dataset

#for refactoring
# AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")
# AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
# AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
# AWS_SESSION_TOKEN = os.getenv("AWS_SESSION_TOKEN")

AWS_S3_BUCKET = "<s3 bucket>" 
AWS_ACCESS_KEY_ID = "<access key>"
AWS_SECRET_ACCESS_KEY = "<secret key >"
AWS_SESSION_TOKEN = "<session token>"
storage_options= {
            "key": AWS_ACCESS_KEY_ID,
            "secret": AWS_SECRET_ACCESS_KEY,
            "token": AWS_SESSION_TOKEN,
        }

#s3 = datasets.filesystems.S3FileSystem(key=storage_options["key"], secret=storage_options["secret"], token=storage_options["token"])
s3 = S3FileSystem(key=storage_options["key"], secret=storage_options["secret"], token=storage_options["token"])




def create_dataset(file_list):
# Create an empty dataframe to store the combined data
    combined_df = pd.DataFrame()

    # Loop through each CSV file and append its contents to the combined dataframe
    for csv_file in file_list:
        print(csv_file)
        df=pd.read_csv(
            f"s3://{csv_file}",
            storage_options=storage_options,
        )
        print(df.info())
        combined_df = pd.concat([combined_df, df], ignore_index=True)

    print(combined_df.info())
    
    return Dataset.from_pandas(combined_df)

csv_files=s3.ls("s3://goes-se-sandbox01/vishr/data/FINDSum/FINDSum-ROO/train")
# remove the first item which contains the directory
csv_files.pop(0)


print(f"train files : \n {csv_files}")
train_ds = create_dataset(csv_files)
print(f"train dataset: \n {train_ds} ")
#next use test csv files
csv_files=s3.ls("s3://goes-se-sandbox01/vishr/data/FINDSum/FINDSum-ROO/test")
# remove the first item which contains the directory
csv_files.pop(0)
print(f"test files : \n {csv_files}")
test_ds = create_dataset(csv_files)
print(f"train dataset: \n {test_ds} ")

/home/cdsw/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


train files : 
 ['goes-se-sandbox01/vishr/data/FINDSum/FINDSum-ROO/train/train_roo_segment_0_input_2_1000.csv', 'goes-se-sandbox01/vishr/data/FINDSum/FINDSum-ROO/train/train_roo_segment_1_input_2_1000.csv']
goes-se-sandbox01/vishr/data/FINDSum/FINDSum-ROO/train/train_roo_segment_0_input_2_1000.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16820 entries, 0 to 16819
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  16820 non-null  object
 1   summary   16820 non-null  object
dtypes: object(2)
memory usage: 262.9+ KB
None
goes-se-sandbox01/vishr/data/FINDSum/FINDSum-ROO/train/train_roo_segment_1_input_2_1000.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16820 entries, 0 to 16819
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  16820 non-null  object
 1   summary   16820 non-null  object
dtypes: object(2)
memory usage: 262.9+ KB


In [5]:
#set up the train and test classes
from datasets import DatasetDict

ds_fin= DatasetDict()
ds_fin["train"]=  train_ds
ds_fin["test"]=test_ds


In [6]:
# let us have a look at our dataset
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(ds_fin["train"], 2)

In [8]:

print(f"Train dataset size: {len(ds_fin['train'])}")
print(f"Test dataset size: {len(ds_fin['test'])}")


Train dataset size: 33640
Test dataset size: 4204


In [9]:
#!pip install tensorflow keras  transformers torch --quiet


In [10]:
#!pip install tensorflow==2.8.0

In [11]:
# Foundational model output 1 : Summarization
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="t5-small"
sample_record = ds_fin["train"][0]
sample="summarize: " + sample_record["document"]
tokenizer = AutoTokenizer.from_pretrained(model_id)
input_tokens = tokenizer(sample, padding='max_length', max_length=512, truncation=True, return_tensors='pt')
model_id="t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
# let us save the model
model.save_pretrained('./models/base')
result_sample = model.generate(**input_tokens, max_length=200, top_k=3, temperature=0.5)
input_tokens = tokenizer(sample, padding='max_length', max_length=512, truncation=True, return_tensors='pt')
print(tokenizer.decode(result_sample[0], truncate_before_pattern=[r"\n\n^#", "^'''", "\n\n\n"]))

<pad> lcfh's financial statements reflect our plans, estimates and beliefs. the forward-looking statements could differ materially from those discussed in the forward-looking statements. we are a leading commercial real estate finance company with a proprietary loan origination platform and an established national footprint.</s>


In [12]:
# Foundational model output 2 : Translation
input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text,  max_length=512,truncation=True,  return_tensors="pt")

outputs = model.generate(**input_ids, max_length=200, top_k=3, temperature=0.5)
print(tokenizer.decode(outputs[0]))

<pad> Wie alt sind Sie?</s>


To train our model, we need to convert our inputs (text) to token IDs.

Before we can start training, we need to preprocess our data. Abstractive Summarization is a text-generation task. Our model will take a text as input and generate a summary as output. We want to understand how long our input and output will take to batch our data efficiently.

In [13]:
# Vish
from datasets import concatenate_datasets
import numpy as np
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([ds_fin["train"], ds_fin["test"]]).map(lambda x: tokenizer(x["document"], truncation=True), batched=True, remove_columns=["document", "summary"])
input_lenghts = [len(x) for x in tokenized_inputs["input_ids"]]
# take 85 percentile of max length for better utilization
max_source_length = int(np.percentile(input_lenghts, 85))
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([ds_fin["train"], ds_fin["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["document", "summary"])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# take 90 percentile of max length for better utilization
max_target_length = int(np.percentile(target_lenghts, 90))
print(f"Max target length: {max_target_length}")

Map: 100%|███████████████████████| 37844/37844 [00:31<00:00, 1186.42 examples/s]


Max source length: 512


Map: 100%|███████████████████████| 37844/37844 [00:09<00:00, 4132.35 examples/s]


Max target length: 512


<b>We preprocess our dataset before training and save it to disk.

In [14]:
#vish
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["document"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = ds_fin.map(preprocess_function, batched=True, remove_columns=["document", "summary"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

# save datasets to disk for later easy loading/ For very large Datasets, we can even save it to the Datalake or an S3 Bucket
tokenized_dataset["train"].save_to_disk("data/train")
tokenized_dataset["test"].save_to_disk("data/eval")

Map: 100%|██████████████████████████| 4204/4204 [00:06<00:00, 642.12 examples/s]


Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


Saving the dataset (1/1 shards): 100%|█| 33640/33640 [00:02<00:00, 14679.19 exam
Saving the dataset (1/1 shards): 100%|█| 4204/4204 [00:00<00:00, 17459.17 exampl


## 3. Fine-Tune T5 with LoRA and bnb int-8

In addition to the LoRA technique, we will use [bitsanbytes LLM.int8()](https://huggingface.co/blog/hf-bitsandbytes-integration) to quantize out frozen LLM to int8. This allows us to reduce the needed memory for FLAN-T5 XXL ~4x.  

The first step of our training is to load the model. We are going to use [philschmid/flan-t5-xxl-sharded-fp16](https://huggingface.co/philschmid/flan-t5-xxl-sharded-fp16), which is a sharded version of [google/flan-t5-xxl](https://huggingface.co/google/flan-t5-xxl). The sharding will help us to not run off of memory when loading the model.

In [15]:
from transformers import AutoModelForSeq2SeqLM
import torch

#Vish
model_id = "t5-small"

# model_id = "google/flan-t5-xl"

# with torch.autocast("cuda"):
  # load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto")


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/cdsw/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/cdsw/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/cuda/lib'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)


Now, we can prepare our model for the LoRA int-8 training using `peft`.

In [16]:


from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# Define LoRA Config
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)
# prepare int-8 model for training - Commenting Vish
model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# trainable params: 18874368 || all params: 11154206720 || trainable%: 0.16921300163961817

trainable params: 589824 || all params: 61096448 || trainable%: 0.9653981848502878


### PEFT Training:
As you can see, here we are only training 0.97% of the parameters of the model! This huge memory gain will enable us to fine-tune the model without memory issues.
Next is to create a `DataCollator` that will take care of padding our inputs and labels. We will use the `DataCollatorForSeq2Seq` from the 🤗 Transformers library.

In [17]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

The last step is to define the hyperparameters (`TrainingArguments`) we want to use for our training.

In [18]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir="lora-flan-t5-small"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=2,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="no",
    report_to="tensorboard",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

Let's now train our model and run the cells below. Note that for T5, some layers are kept in `float32` for stability purposes.

In [19]:
#  with torch.autocast("cuda"):
trainer.train(resume_from_checkpoint = True)

/home/cdsw/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/cdsw/.local/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
500,3.995400
1000,3.731100
1500,3.629600
2000,3.579700
2500,3.532100
3000,3.481700
3500,3.481700
4000,3.492100
4500,3.446000
5000,3.439300


TrainOutput(global_step=8410, training_loss=3.511894293205633, metrics={'train_runtime': 6316.9204, 'train_samples_per_second': 10.651, 'train_steps_per_second': 1.331, 'total_flos': 9227703681024000.0, 'train_loss': 3.511894293205633, 'epoch': 2.0})

The training took ~10:36:00 and cost `~13.22$` for 10h of training. For comparison a [full fine-tuning on FLAN-T5-XXL](https://www.philschmid.de/fine-tune-flan-t5-deepspeed#3-results--experiments) with the same duration (10h) requires 8x A100 40GBs and costs ~322$.

We can save our model to use it for inference and evaluate it. We will save it to disk for now

In [20]:
# Save our LoRA model & tokenizer results
peft_model_id="results"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)


('results/tokenizer_config.json',
 'results/special_tokens_map.json',
 'results/tokenizer.json')

In [21]:
# if you want to save the base model to call
peft_model_id="results_base"
trainer.model.base_model.save_pretrained(peft_model_id)

/home/cdsw/.local/lib/python3.9/site-packages/transformers/modeling_utils.py:1662: UserWarning: You are calling `save_pretrained` to a 8-bit converted model you may likely encounter unexepected behaviors. 
  warnings.warn(


Our LoRA checkpoint is only 84MB small and includes all of the learnt knowleddge for samsum.

## 4. Evaluate & run Inference with LoRA FLAN-T5

After the training is done we want to evaluate and test it. The most commonly used metric to evaluate summarization task is [rogue_score](https://en.wikipedia.org/wiki/ROUGE_(metric)) short for Recall-Oriented Understudy for Gisting Evaluation). This metric does not behave like the standard accuracy: it will compare a generated summary against a set of reference summaries.

We are going to use `evaluate` library to evaluate the `rogue` score. We can run inference using `PEFT` and `transformers`. For our FLAN-T5 XXL model, we need at least 18GB of GPU memory.

In [22]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "results"
config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,  load_in_8bit=True,  device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

print("Peft model loaded")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Peft model loaded


In [23]:
#Vish : Let us look at an unseen data from Train dataset
unseen_dataset=  ds_fin["test"]

In [26]:
show_random_elements(unseen_dataset, 1)

In [27]:
import random
sample_data= unseen_dataset[random.randrange(len(unseen_dataset))]

In [28]:
sample_data["document"]

"as a result of the execution of the settlement agreement , a stipulated motion for dismissal with prejudice was filed with the court which includes a form of order of dismissal with prejudice ( the “ court order ” ) . on may 15 , 2017 , the court order was executed by the judge and the suit was formally dismissed with prejudice . note 10. related party transactions notes payable in 2014 , the company issued notes payable to related parties in the amount of $ 2 million . the notes bear interest at 7.5 % and were scheduled to mature on january 2 , 2017. the company did not pay these notes upon maturity as the company and the related parties informally agreed to offset these notes payable with the related-party note receivable . during the year , the company made principal payments and interest payments of $ 80,000 related to the notes payable . additionally , the company applied $ 207,942 in principal and interest due to the company on the related party note receivable ( see note 6 – re

In [29]:
input_ids = tokenizer(sample_data["document"], return_tensors="pt", truncation=True).input_ids.cuda()
#with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens= 100, do_sample=True, top_p=0.8)
print(f"input sentence: {sample_data['document']}\n{'---'* 20}")

print(f"summary:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")

input sentence: as a result of the execution of the settlement agreement , a stipulated motion for dismissal with prejudice was filed with the court which includes a form of order of dismissal with prejudice ( the “ court order ” ) . on may 15 , 2017 , the court order was executed by the judge and the suit was formally dismissed with prejudice . note 10. related party transactions notes payable in 2014 , the company issued notes payable to related parties in the amount of $ 2 million . the notes bear interest at 7.5 % and were scheduled to mature on january 2 , 2017. the company did not pay these notes upon maturity as the company and the related parties informally agreed to offset these notes payable with the related-party note receivable . during the year , the company made principal payments and interest payments of $ 80,000 related to the notes payable . additionally , the company applied $ 207,942 in principal and interest due to the company on the related party note receivable ( 

Nice! our model works! Now, lets take a closer look and evaluate it against the `test` set of processed dataset from `samsum`. Therefore we need to use and create some utilities to generate the summaries and group them together. The most commonly used metrics to evaluate summarization task is [rogue_score](https://en.wikipedia.org/wiki/ROUGE_(metric)) short for Recall-Oriented Understudy for Gisting Evaluation). This metric does not behave like the standard accuracy: it will compare a generated summary against a set of reference summaries.

In [ ]:
import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# Metric
metric = evaluate.load("rouge")

def evaluate_peft_model(sample,max_target_length=50):
    # generate summary
    outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
    # decode eval sample
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    labels = tokenizer.decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    return prediction, labels

# load test dataset from distk
test_dataset = load_from_disk("data/eval/").with_format("torch")

# run predictions
# this can take ~45 minutes
predictions, references = [] , []
for sample in tqdm(test_dataset):
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append(l)

# compute metric
rogue = metric.compute(predictions=predictions, references=references, use_stemmer=True)

# print results
print(f"Rogue1: {rogue['rouge1']* 100:2f}%")
print(f"rouge2: {rogue['rouge2']* 100:2f}%")
print(f"rougeL: {rogue['rougeL']* 100:2f}%")
print(f"rougeLsum: {rogue['rougeLsum']* 100:2f}%")

# Rogue1: 50.386161%
# rouge2: 24.842412%
# rougeL: 41.370130%
# rougeLsum: 41.394230%

  3%|█                                     | 112/4204 [04:06<2:29:46,  2.20s/it]

Our PEFT fine-tuned FLAN-T5-XXL achieved a rogue1 score of `50.38%` on the test dataset. For comparison a [full fine-tuning of flan-t5-base achieved a rouge1 score of 47.23](https://www.philschmid.de/fine-tune-flan-t5). That is a `3%` improvements.

It is incredible to see that our LoRA checkpoint is only 84MB small and model achieves better performance than a smaller fully fine-tuned model.